In [ ]:
from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.utilities.plotting_utilities import plot_errorbar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# notebook to showing how to construct AdlerPlanetoid from the MPC obs_sbn/SSSC Prompt Products Bandaid

In [ ]:
# ssObjectId of object to analyse
ssoid1 = "2025 MX40"  # test object
ssoid2 = "2025 MX40Fake"  # fake outburst object
filt = "r"

In [ ]:
# here we use an offline SQL database which contains the observations of the sso
# fname = "/home/jrob/lsst-adler/notebooks/gen_test_data/adler_demo_testing_database.db"
fname = (
    "/Users/seanobrien/Documents/Adler/lsst-adler/notebooks/gen_test_data/mpc_obs_sbn_testing_database.sqlite"
)

# If running on the larger file and the information on topocentricDist, heliocentricDist, phaseAngle have not been populated, you can run this function
# However! The csv file it calls must be constructed by querying JPL Horizons for each observation and that isn't something we wish to encourage as it will overload their service
# #from adler.objectdata.objectdata_utilities import mpc_file_preprocessing
# #mpc_file_preprocessing(sql_filename=fname, jplhorizons_filename="jplhorizons_data.csv")

planetoid1 = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssoid1, sql_filename=fname)
planetoid2 = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssoid2, sql_filename=fname)

# There is no RSP analog currently
# Perhaps in future the sqlite file will be hosted in a shared location on the RSP

In [ ]:
planetoid1

In [ ]:
planetoid1.__dict__

In [ ]:
fig = plot_errorbar(planetoid1, filt_list=["g"], x_plot="midPointMjdTai", y_plot="reduced_mag")
fig = plot_errorbar(planetoid1, filt_list=["g"], x_plot="phaseAngle", y_plot="reduced_mag")

In [ ]:
fig = plot_errorbar(planetoid2, filt_list=["g"], x_plot="midPointMjdTai", y_plot="reduced_mag")
fig = plot_errorbar(planetoid2, filt_list=["g"], x_plot="phaseAngle", y_plot="reduced_mag")

In [ ]:
# inspect observations

In [ ]:
obs = planetoid1.observations_in_filter(filt)

In [ ]:
df_obs = pd.DataFrame(obs.__dict__)

In [ ]:
df_obs

In [ ]:
tmin = np.amin(np.floor(df_obs["midPointMjdTai"]))  # mjd
tmax = np.amax(np.floor(df_obs["midPointMjdTai"])) + 1  # mjd
tmin, tmax

In [ ]:
# cumulative data in filter
x_plot = "midPointMjdTai"
df_plot = df_obs.sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

bins = np.arange(tmin, tmax + 1)

values, base = np.histogram(df_plot[x_plot], bins=bins)
cumulative = np.cumsum(values)
ax1.plot(base[:-1] - base[0], cumulative, label=filt)

data_mask = np.diff(cumulative) > 0
data_nights = base[1:-1][data_mask]
N_data = cumulative[1:][data_mask]

ax1.scatter(data_nights - data_nights[0], N_data)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("number")
ax1.legend()

plt.show()

In [ ]:
# number of data points per night of new data
np.diff(N_data)

In [ ]:
# nights when new data arrives
data_nights